## nanoGPT trained on Harry Potter Dataset

### Problem  Statemet

1. Creating a character-level GPT on the books of Harry Potter.
2. Fine-tuning GPT-2(created by Karpathy) on the books of Harry Potter.

This project is based on the Karpathy's work - https://github.com/karpathy/nanoGPT

### The Dataset

The dataset was downloaded from - https://github.com/priya-dwivedi/Deep-Learning/tree/master/GPT2-HarryPotter-Training/books

All files were then merged and fed into the model.

In [1]:
# Reading the file
with open('harry_potter_series.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  6765168


In [3]:
# let's look at the first 1000 characters
print(text[:1000])

/ 




THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, 
were proud to say that they were perfectly normal, 
thank you very much. They were the last people you’d 
expect to be involved in anything strange or 
mysterious, because they just didn’t hold with such 
nonsense. 

Mr. Dursley was the director of a firm called 
Grunnings, which made drills. He was a big, beefy 
man with hardly any neck, although he did have a 
very large mustache. Mrs. Dursley was thin and 
blonde and had nearly twice the usual amount of 
neck, which came in very useful as she spent so 
much of her time craning over garden fences, spying 
on the neighbors. The Dursley s had a small son 
called Dudley and in their opinion there was no finer 
boy anywhere. 

The Dursleys had everything they wanted, but they 
also had a secret, and their greatest fear was that 
somebody would discover it. They didn’t think they 
could bear it if anyone found out about the Potters. 
Mrs. Potter was Mrs. Dursl

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"%&'()*,-./0123456789:;>?ABCDEFGHIJKLMNOPQRSTUVWXYZ\]abcdefghijklmnopqrstuvwxyz|~—‘’“”•■□
92


### Methodology

### Results

The following graphs were created on https://wandb.ai/ platform. 

We used this platform to keep track of our hyperparameters, system metrics, predictions and losses

#### Graphs for Character-level GPT:

**A. Iteration vs Step**
<img src="harrypotter_char_iter.png" alt="Iteration vs Step" />

**B. Train loss vs Step**
<img src="harrypotter_char_train_loss.png" alt="Train loss vs Step" />

**C. Val loss vs Step**
<img src="harrypotter_char_val_loss.png" alt="Val loss vs Step" />

#### Graphs for GPT-2:

**A. Iteration vs Step**
<img src="harrypotter_gpt2_iter.png" alt="Iteration vs Step" />

**B. Train loss vs Step**
<img src="harrypotter_gpt2_train_loss.png" alt="Train loss vs Step" />

**C. Val loss vs Step**
<img src="harrypotter_gpt2_val_loss.png" alt="Val loss vs Step" />

### Conclusion

### Refrences

https://github.com/karpathy/nanoGPT

https://github.com/priya-dwivedi/Deep-Learning/tree/master/GPT2-HarryPotter-Training/books